In [2]:
# !pip install selenium
# !pip install beautifulsoup4

  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
  Attempting uninstall: certifi
    Found existing installation: certifi 2020.12.5
    Uninstalling certifi-2020.12.5:
      Successfully uninstalled certifi-2020.12.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.


In [3]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm

In [6]:
#path to the webdriver file
service = webdriver.ChromeService(executable_path = 'chromedriver.exe')
driver = webdriver.Chrome(service=service)
url = 'https://www.imdb.com/search/title/?groups=top_100'
driver.get(url)


In [7]:
# Set initial empty list for each element:
title = []
link = []
# Grab the block of each individual movie
blocks = driver.find_elements(By.CLASS_NAME, "ipc-metadata-list-summary-item")

# Set up a for loop to run through all movies
for i in range(0,50):
    # Extracting title
    ftitle = blocks[i].find_element(By.CLASS_NAME, 'ipc-title__text').text

    # Then extract the link with cleaned title
    flink = blocks[i].find_element(By.LINK_TEXT, ftitle).get_attribute('href')

    # Add item to the respective lists
    title.append(ftitle)
    link.append(flink)


In [8]:
user_review_links = []
for url in link:
    try:
        #setup user agent for BS4, except some rare case, it would be the same for most browser 
        user_agent = {'User-agent': 'Mozilla/5.0'}
        #Use request.get to load the whole page
        response = requests.get(url, headers = user_agent)
        #Parse the requests object to BS4 to transform it into html structure
        soup = BeautifulSoup(response.text, 'html.parser')
        #Find the link marked by the USER REVIEWS link text.
        review_link = 'https://www.imdb.com'+soup.find('a', text = 'User reviews').get('href')
        #Append the newly grabed link into its list
        user_review_links.append(review_link)
    except:
            print(url)

In [9]:
len(user_review_links)

50

In [39]:
replacements = {'\n': '', '*': '', '\\': ''}

movie_title = []
review_title = []
content = []
rating = []
index = 0

for review_link in user_review_links:

    review_driver = webdriver.Chrome(service=service)
    review_driver.get(review_link)

    fmovie_title = title[index]
    index += 1

    blocks = review_driver.find_elements(By.CLASS_NAME, "lister-item-content")

    for i in range(len(blocks)):
        try:
            # Extracting title
            ftitle = blocks[i].find_element(By.CLASS_NAME, 'title').text

            fcontent = blocks[i].find_element(By.CLASS_NAME, 'content').text
            replaced_chars = [replacements.get(char, char) for char in fcontent[:-87]]
            result = ''.join(replaced_chars)

            fratting = (blocks[i].find_element(By.CLASS_NAME, 'rating-other-user-rating').text)[:-3]

            # Add item to the respective lists
            movie_title.append(fmovie_title)
            review_title.append(ftitle)
            content.append(result)
            rating.append(fratting)
        except:
            continue
    review_driver.quit()
    

# Export to csv file

In [40]:
#Build data dictionary for dataframe
data = {'Movie_name': movie_title, 
        'Review Rating': rating,
        'Review title': review_title, 
        'Review_content' : content
   }
#Build dataframe to export
review = pd.DataFrame(data = data)

In [41]:
review.head()

,Movie_name,Review Rating,Review title,Review_content
0,1. Oppenheimer,8,"A challenging watch to be sure, but a worthwhi...",
1,1. Oppenheimer,9,Murphy is exceptional,You'll have to have your wits about you and yo...
2,1. Oppenheimer,7,Quality but exhausting,I'm a big fan of Nolan's work so was really lo...
3,1. Oppenheimer,10,A brilliantly layered examination of a man thr...,"""Oppenheimer"" is a biographical thriller film ..."
4,1. Oppenheimer,10,Nolan delivers a powerfull biopic that shows t...,This movie is just... wow! I don't think I hav...


In [42]:
review.tail()

,Movie_name,Review Rating,Review title,Review_content
1105,50. Casablanca,10,"""Who's got trouble? We got trouble.""","For the Allies, 1943 was perhaps the grimmest ..."
1106,50. Casablanca,10,We'll Always Have Casablanca,"Spoilers ahead, but then again, who isn't fami..."
1107,50. Casablanca,10,"For all-around entertainment, the best I've ev...",
1108,50. Casablanca,10,NEVER FORGET ALL THESE GREAT WARNER BROTHERS P...,"They're all gone now, but you have to tip your..."
1109,50. Casablanca,10,Rightfully one of cinema's finest stories ever...,"""Casablanca"" is a timely and timeless classic ..."


In [44]:
review.to_csv('csv_result/imdb_top_movie_review.csv',index=False)

In [45]:
link_data = {'Movie_name': title, 
        'Movie link': link,
        'Review link': user_review_links
   }
#Build dataframe to export
moive_review_link = pd.DataFrame(data = link_data)

In [48]:
moive_review_link.head()

,Movie_name,Movie link,Review link
0,1. Oppenheimer,https://www.imdb.com/title/tt15398776/?ref_=sr...,https://www.imdb.com/title/tt15398776/reviews/...
1,2. It's a Wonderful Life,https://www.imdb.com/title/tt0038650/?ref_=sr_t_2,https://www.imdb.com/title/tt0038650/reviews/?...
2,3. Parasite,https://www.imdb.com/title/tt6751668/?ref_=sr_t_3,https://www.imdb.com/title/tt6751668/reviews/?...
3,4. Spider-Man: Across the Spider-Verse,https://www.imdb.com/title/tt9362722/?ref_=sr_t_4,https://www.imdb.com/title/tt9362722/reviews/?...
4,5. The Shawshank Redemption,https://www.imdb.com/title/tt0111161/?ref_=sr_t_5,https://www.imdb.com/title/tt0111161/reviews/?...


In [49]:
moive_review_link.to_csv('csv_result/imdb_top_movie_link.csv',index=False)